In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, END, START
from typing import Annotated,Sequence, Literal,TypedDict
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv

c:\Users\omarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()


True

In [ ]:
embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
db = Chroma

In [ ]:
from langgraph.graph import StateGraph, END

from typing import TypedDict, Optional, List
import re

########################################
# State Definition
########################################

class State(TypedDict):
    raw_text: str
    header_text: str
    body_text: str
    heuristic_type: Optional[str]
    heuristic_confidence: int
    llm_type: Optional[str]
    llm_confidence: Optional[int]
    final_type: Optional[str]
    explanation: Optional[str]


########################################
# Node: Preprocess
########################################

def preprocess_node(state: State):

    text = state["raw_text"]

    # Normalize Arabic forms (very basic)
    text = re.sub(r"[ـ]+", "", text)  # remove tatweel
    text = re.sub(r"\s+", " ", text)

    # Split header (first 10 lines)
    lines = text.strip().split("\n")
    header = "\n".join(lines[:6])
    body = "\n".join(lines[6:])

    state["header_text"] = header
    state["body_text"] = body
    return state


########################################
# Node: Header Keyword Classifier
########################################

DOC_KEYWORDS = {
    "صحيفة دعوى": ["صحيفة دعوى", "أقامت المدعية", "وأقامت هذه الدعوى"],
    "مذكرة دفاع": ["مذكرة بدفاع", "مذكرة مقدمة من"],
    "مذكرة رد": ["مذكرة بالرد", "رداً على مذكرة"],
    "إعلان": ["إعلان", "يعلن السيد", "إعلان رسمي"],
    "حكم": ["حكم", "بعد الاطلاع على الأوراق", "وتداولت المحكمة"],
    "محضر جلسة": ["محضر جلسة", "انعقدت الجلسة"],
    "مستند": ["مستند", "مرفق", "مستند رقم"]
}

def header_classifier_node(state: State):

    header = state["header_text"]
    for doc_type, keywords in DOC_KEYWORDS.items():
        for kw in keywords:
            if kw in header:
                state["heuristic_type"] = doc_type
                state["heuristic_confidence"] = 90
                return state

    # No match
    state["heuristic_type"] = None
    state["heuristic_confidence"] = 0
    return state


########################################
# Router Node: Check Heuristic Confidence
########################################

def check_confidence_node(state: State):
    return state

def confidence_router(state: State):
    if state["heuristic_confidence"] >= 70:
        state["final_type"] = state["heuristic_type"]
        state["explanation"] = "Identified via header keyword match."
        return "use_heuristic"
    return "use_llm"


########################################
# Node: LLM Semantic Classifier
########################################

LLM = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

def llm_semantic_classifier_node(state: State):

    HEADER = state["header_text"]
    BODY = state["body_text"][:2000]   # limit context for speed

    prompt = f"""
أنت خبير قانوني متخصص في مستندات القضاء المدني المصري.
مهمتك تحديد نوع المستند بدقة شديدة.

النص التالي هو مستند قانوني. قم بتحليل:
- العبارات القانونية
- المتحدث (قاضي / محامٍ)
- البنية (طلبات، دفوع، حكم، تقرير جلسة)
- أي إشارات لمواد القانون

أعد النتيجة بصيغة JSON فقط.

المستند:
---
{HEADER}
====
{BODY}
---

قائمة الأنواع المسموح بها:
["صحيفة دعوى", "مذكرة دفاع", "مذكرة رد", "إعلان", "حكم", "محضر جلسة", "مستند", "غير ذلك"]

أعد JSON بهذا الشكل فقط:

{{
  "doc_type": "...",
  "confidence": 0-100,
  "reasons": "..."
}}
"""

    result = LLM.invoke(prompt)
    import json
    parsed = json.loads(result.content)

    state["llm_type"] = parsed["doc_type"]
    state["llm_confidence"] = parsed["confidence"]
    state["explanation"] = parsed["reasons"]
    state["final_type"] = parsed["doc_type"]
    return state


########################################
# Build Graph
########################################

graph = StateGraph(State)

graph.add_node("preprocess", preprocess_node)
graph.add_node("header_classifier", header_classifier_node)
graph.add_node("check_confidence", check_confidence_node)
graph.add_node("llm_classifier", llm_semantic_classifier_node)

graph.set_entry_point("preprocess")

graph.add_edge("preprocess", "header_classifier")
graph.add_edge("header_classifier", "check_confidence")

graph.add_conditional_edges(
    "check_confidence",
    confidence_router,
    {
        "use_heuristic": END,
        "use_llm": "llm_classifier"
    }
)

graph.add_edge("llm_classifier", END)

app = graph.compile()

########################################
# Run Example
########################################

if __name__ == "__main__":

    sample_text = """
    مذكرة بدفاع السيد/ أحمد
    ضد
    شركة النور

    مقدمة لسيادتكم…
    """

    state = {"raw_text": sample_text}
    result = app.invoke(state)
    print(result)


{'raw_text': '\n    مذكرة بدفاع السيد/ أحمد\n    ضد\n    شركة النور\n\n    مقدمة لسيادتكم…\n    ', 'header_text': 'مذكرة بدفاع السيد/ أحمد ضد شركة النور مقدمة لسيادتكم…', 'body_text': '', 'heuristic_type': 'مذكرة دفاع', 'heuristic_confidence': 90}


In [8]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, Dict, List


class State(TypedDict):
    text: str
    header: str
    body_excerpt: str

    heuristic_type: Optional[str]
    heuristic_confidence: int
    matched_keywords: List[str]

    final_type: Optional[str]
    confidence: int
    explanation: Optional[str]


# --------------------------------------
# 1) Extract header
# --------------------------------------
def extract_header_node(state: State):
    lines = state["text"].split("\n")
    header = "\n".join(lines[:6])
    body_excerpt = " ".join(state["text"].split()[:400])

    state["header"] = header
    state["body_excerpt"] = body_excerpt
    return state


# --------------------------------------
# 2) Heuristic classifier
# --------------------------------------
KEYWORDS = {
    "صحيفة دعوى": ["صحيفة دعوى", "الطلبات", "الوقائع", "بناءً عليه"],
    "مذكرة بدفاع": ["مذكرة بدفاع", "الدفاع", "أولاً", "ثانياً"],
    "حكم": ["باسم الشعب", "فلهذه الأسباب", "قضت المحكمة", "وحيث إن"],
    "محضر جلسة": ["انعقدت المحكمة", "أثبت الحاضرون", "قررت المحكمة"],
    "إعلان": ["إنه في يوم", "أعلنت", "كلفته الحضور"],
    "أمر أداء": ["أمر أداء", "يأمر", "دين ثابت"],
    "أمر على عريضة": ["أمر على عريضة", "بعد الاطلاع", "نأمر"],
    "تقرير خبير": ["تقرير الخبير", "المعاينة", "الخبير المنتدب"],
    "محضر إثبات حالة": ["إثبات حالة", "قام المحضر", "ثبت لديه"],
    "مستند غير معروف": []
}


def heuristic_node(state: State):
    text = state["header"] + "\n" + state["body_excerpt"]
    best_type = None
    best_score = 0
    best_keywords = []

    for doc_type, keys in KEYWORDS.items():
        matches = [k for k in keys if k in text]
        score = len(matches) * 25  # each match adds 25%

        if score > best_score:
            best_score = score
            best_type = doc_type
            best_keywords = matches

    state["heuristic_type"] = best_type
    state["heuristic_confidence"] = min(best_score, 100)
    state["matched_keywords"] = best_keywords
    state["explanation"] = "Heuristic-based classification"

    return state


# --------------------------------------
# 3) Confidence check (returns state only)
# --------------------------------------
def check_confidence_node(state: State):
    return state


# Router
def confidence_router(state: State):
    if state["heuristic_confidence"] >= 70:
        state["final_type"] = state["heuristic_type"]
        state["confidence"] = state["heuristic_confidence"]
        state["explanation"] = (
            "تم تحديد النوع بناءً على الكلمات المفتاحية: "
            + ", ".join(state["matched_keywords"])
        )
        return "use_heuristic"
    return "use_llm"


# --------------------------------------
# 4) LLM classifier
# --------------------------------------
from langchain_openai import ChatOpenAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")


def llm_classifier(state: State):
    prompt = f"""
    صنّف نوع هذا المستند المدني المصري بدقة شديدة.
    النص:
    {state["header"]}\n\n{state["body_excerpt"]}

    الأنواع المحتملة:
    {list(KEYWORDS.keys())}

    أرجع:
    - النوع
    - سبب التصنيف
    - درجة الثقة (0–100)
    """

    result = llm.invoke(prompt).content

    state["final_type"] = result
    state["confidence"] = 80
    state["explanation"] = "LLM-based classification"
    return state


# --------------------------------------
# 5) Build graph
# --------------------------------------
graph = StateGraph(State)

graph.add_node("extract_header", extract_header_node)
graph.add_node("heuristic", heuristic_node)
graph.add_node("check_confidence", check_confidence_node)
graph.add_node("llm_classifier", llm_classifier)

graph.set_entry_point("extract_header")

graph.add_edge("extract_header", "heuristic")
graph.add_edge("heuristic", "check_confidence")

graph.add_conditional_edges(
    "check_confidence",
    confidence_router,
    {
        "use_heuristic": END,
        "use_llm": "llm_classifier",
    },
)

graph.add_edge("llm_classifier", END)

workflow = graph.compile()


In [ ]:
sample_text = """صصحيفة دعوى
مقدمة من المدعي: أحمد عبد الرحمن، المقيم في ١٢ شارع التحرير – الدقي.
ضد المدعى عليه: شركة النور للمقاولات، ومقرها ٥ شارع الطيران – مدينة نصر.

محكمة جنوب القاهرة الابتدائية – الدائرة المدنية رقم 15
رقم الدعوى: لم يُقيد بعد

أولاً: الوقائع
بموجب عقد مؤرخ في ١٠/٥/٢٠٢٢ قامت الشركة المدعى عليها بتنفيذ أعمال تشطيب لشقة المدعي،
إلا أنها أخلّت بالتزاماتها بعدم إتمام الأعمال المتفق عليها، ونتج عن ذلك أضرار مادية جسيمة.

ثانياً: السند القانوني
عملاً بالمواد ١٤٧، ١٥٧ من القانون المدني التي تقضي بفسخ العقد عند إخلال أحد المتعاقدين.

الطلبات
يلتمس المدعي الحكم بفسخ العقد وإلزام الشركة برد المبالغ المدفوعة وبدفع تعويض قدره خمسون ألف جنيه.

بنــاءً عليه
أعلنّا المدعى عليه وكلفناه بالحضور لجلسة… لاتخاذ اللازم قانوناً.

"""
state = {"text": sample_text}
result = workflow.invoke(state)
print(result)

{'text': 'صصحيفة دعوى\nمقدمة من المدعي: أحمد عبد الرحمن، المقيم في ١٢ شارع التحرير – الدقي.\nضد المدعى عليه: شركة النور للمقاولات، ومقرها ٥ شارع الطيران – مدينة نصر.\n\nمحكمة جنوب القاهرة الابتدائية – الدائرة المدنية رقم 15\nرقم الدعوى: لم يُقيد بعد\n\nأولاً: الوقائع\nبموجب عقد مؤرخ في ١٠/٥/٢٠٢٢ قامت الشركة المدعى عليها بتنفيذ أعمال تشطيب لشقة المدعي،\nإلا أنها أخلّت بالتزاماتها بعدم إتمام الأعمال المتفق عليها، ونتج عن ذلك أضرار مادية جسيمة.\n\nثانياً: السند القانوني\nعملاً بالمواد ١٤٧، ١٥٧ من القانون المدني التي تقضي بفسخ العقد عند إخلال أحد المتعاقدين.\n\nالطلبات\nيلتمس المدعي الحكم بفسخ العقد وإلزام الشركة برد المبالغ المدفوعة وبدفع تعويض قدره خمسون ألف جنيه.\n\nبنــاءً عليه\nأعلنّا المدعى عليه وكلفناه بالحضور لجلسة… لاتخاذ اللازم قانوناً.\n\n', 'header': 'صصحيفة دعوى\nمقدمة من المدعي: أحمد عبد الرحمن، المقيم في ١٢ شارع التحرير – الدقي.\nضد المدعى عليه: شركة النور للمقاولات، ومقرها ٥ شارع الطيران – مدينة نصر.\n\nمحكمة جنوب القاهرة الابتدائية – الدائرة المدنية رقم 15\nرقم الدعوى: لم 

: 